In [1]:
import numpy as np
import pandas as pd

dna1 = ' ' + input()  # пробел для пустого столбца в таблице
dna2 = ' ' + input()

n = len(dna1)  # число столбцов
m = len(dna2)  # число строк
d = 10  # штраф за гэп
e = 0.5

# array для scores
scores = np.zeros((m, n))
# заполняем первую строку
scores[0][0] = 0
scores[0, 1:] = [-10 - 0.5 * i for i in range(n - 1)]
# заполняем первый столбец
scores[1:, 0] = [-10 - 0.5 * i for i in range(m - 1)]

# array для стрелочек
directions = np.array(['//.'] * m * n).reshape(m, n)
# заполняем первую строку
directions[0] = np.array(['→'] * n)
# заполняем первый столбец
directions[:, 0] = np.array(['↓'] * m)

# создаем матрицу А
matrix_A = np.zeros((m - 1, n - 1))
# заполняем первую строку
matrix_A[0, :] = [-20 - 0.5 * i for i in range(n - 1)]

# создаем матрицу В
matrix_B = np.zeros((m - 1, n - 1))
# заполняем первый столбец
matrix_B[:, 0] = [-20 - 0.5 * i for i in range(m - 1)]


def dna_full(n1, n2):
    if n1 == n2:
        return 5
    return -4


# делаем data frame в pandas, чтобы обращаться к нуклеотидам
# в названиях столбцов и строк
df_pd = pd.DataFrame(scores, columns=list(dna1), index=list(dna2))

# заполянем scores, matrix A, matrix B
for i in range(m - 1):
    for j in range(n - 1):
        if i != 0:
            matrix_A[i][j] = max(matrix_A[i - 1][j] - e, scores[i][j + 1] - d)
        if j != 0:
            matrix_B[i][j] = max(matrix_B[i][j - 1] - e, scores[i + 1][j] - d)
        maximum = max(matrix_A[i][j], matrix_B[i][j],
                      scores[i][j] + dna_full(df_pd.columns[j + 1],
                                              df_pd.index[i + 1]))
        scores[i + 1][j + 1] = maximum
        # заполняем directions, учитывая случаи, когда есть 2 равных по стоимости варианта
        if maximum == matrix_B[i][j] and maximum == matrix_A[i][j]:
            directions[i + 1][j + 1] = '→↓'
        elif maximum == matrix_B[i][j] and \
                maximum == scores[i][j] + \
                dna_full(df_pd.columns[j + 1], df_pd.index[i + 1]):
            directions[i + 1][j + 1] = '→↘'
        elif maximum == matrix_A[i][j] and maximum == scores[i][j] + dna_full(
                df_pd.columns[j + 1], df_pd.index[i + 1]):
            directions[i + 1][j + 1] = '↓↘'
        elif maximum == scores[i][j] + dna_full(df_pd.columns[j + 1],
                                                df_pd.index[i + 1]):
            directions[i + 1][j + 1] = '↘'
        elif maximum == matrix_B[i][j]:
            directions[i + 1][j + 1] = '→'
        elif maximum == matrix_A[i][j]:
            directions[i + 1][j + 1] = '↓'


# Делаем traceback и восстанавливаем выравнивание по стрелочкам
alignment = [[], []]
i = 0
while m > 1 and n > 1:
    # при пути назад может быть проблема с выбором направления,
    # если в directions парный знак, поэтому отдельно обрабатываем эти случаи
    if m < len(dna2):  # чтобы работало на самой нижней строке
        # при равных score выбирать путь с гэпом, если строкой выше или ниже уже есть гэп
        if directions[m - 1][n - 1] == '↓↘' and (directions[m-2][n-1] == '↓' or directions[m][n-1] == '↓'):
            alignment[0].append('-')
            alignment[1].append(df_pd.index[m - 1])
            m -= 1
        # иначе путь по диагонали
        elif directions[m - 1][n - 1] == '↓↘' and (directions[m-2][n-1] != '↓' or directions[m][n-1] != '↓'):
            alignment[0].append(df_pd.columns[n - 1])
            alignment[1].append(df_pd.index[m - 1])
            n -= 1
            m -= 1
    else:
        # то же самое, но без условий на строку ниже
        if directions[m - 1][n - 1] == '↓↘' and directions[m-2][n-1] == '↓':
            alignment[0].append('-')
            alignment[1].append(df_pd.index[m - 1])
            m -= 1
        elif directions[m - 1][n - 1] == '↓↘' and directions[m-2][n-1] != '↓':
            alignment[0].append(df_pd.columns[n - 1])
            alignment[1].append(df_pd.index[m - 1])
            n -= 1
            m -= 1
    if n < len(dna2):
        # при равных score выбирать путь с гэпом, если столбцом левее или правее уже есть гэп
        if directions[m - 1][n - 1] == '→↘' and (directions[m-1][n-2] == '→' or directions[m-1][n] == '→'):
            alignment[0].append(df_pd.columns[n - 1])
            alignment[1].append('-')
            n -= 1
        # иначе путь по диагонали
        elif directions[m - 1][n - 1] == '→↘' and (directions[m-1][n-2] != '→' or directions[m-1][n] != '→'):
            alignment[0].append(df_pd.columns[n - 1])
            alignment[1].append(df_pd.index[m - 1])
            n -= 1
            m -= 1
    else:
        # то же самое, но без условий на столбец правее
        if directions[m - 1][n - 1] == '→↘' and directions[m-1][n-2] == '→':
            alignment[0].append(df_pd.columns[n - 1])
            alignment[1].append('-')
            n -= 1
        elif directions[m - 1][n - 1] == '→↘' and directions[m-1][n-2] != '→':
            alignment[0].append(df_pd.columns[n - 1])
            alignment[1].append(df_pd.index[m - 1])
            n -= 1
            m -= 1
    if directions[m - 1][n - 1] == '↓':
        alignment[0].append('-')
        alignment[1].append(df_pd.index[m - 1])
        m -= 1
    elif directions[m - 1][n - 1] == '↘':
        alignment[0].append(df_pd.columns[n - 1])
        alignment[1].append(df_pd.index[m - 1])
        n -= 1
        m -= 1
    elif directions[m - 1][n - 1] == '→':
        alignment[0].append(df_pd.columns[n - 1])
        alignment[1].append('-')
        n -= 1
    i += 1

print(f'Score: {scores[-1][-1]}')
print(f'Alignment:\n{"".join(alignment[0][::-1])}\n'
      f'{"".join(alignment[1][::-1])}')


ATGCCGATCA
ACCTAC
Score: -9.0
Alignment:
ATGCCGATCA
A--CCTA--C
